In [1]:
# define some useful functions

def read_bytes(path):
    with open(path, 'rb') as f:
        return f.read()

def write_bytes(path, bytes):
    with open(path, 'wb') as f:
        f.write(bytes)

def print_list(llist):
    for i in llist:
        print(i)


In [2]:
# define png chunk parser

def parse_png_chunk_data_list(bytes):
    output_data_list = []
    i = 0
    if bytes[i:i+8] != b'\x89PNG\r\n\x1a\n':
        raise Exception('invalid PNG header')
    output_data_list.append({
        'type': 'header',
        'start': i,
        'end': i+8,
        'length': 8,
    })
    i += 8
    while True:
        if i >= len(bytes):
            raise Exception(f'unexpected end of file: i={i}, len(bytes)={len(bytes)}')
        start = i
        length = int.from_bytes(bytes[i:i+4], 'big')
        i += 4
        chunk_type = bytes[i:i+4]
        i += 4
        i += length
        i += 4 # CRC
        end = i
        output_data_list.append({
            'type': 'chunk',
            'chunk_type': chunk_type,
            'start': start,
            'end': end,
            'length': end - start,
        })
        if chunk_type == b'IEND':
            break
    return output_data_list


In [3]:
# sample file names

sample_charcard_face_0_fn   = 'HCCP_F_20230913224620491.png'
sample_charcard_face_100_fn = 'HCCP_F_20230913225210929.png'


In [4]:
# sample file bytes and parsed data

sample_charcard_face_0_bytes = read_bytes(sample_charcard_face_0_fn)
sample_charcard_face_0_chunk_data_list = parse_png_chunk_data_list(sample_charcard_face_0_bytes)
print_list(sample_charcard_face_0_chunk_data_list)


{'type': 'header', 'start': 0, 'end': 8, 'length': 8}
{'type': 'chunk', 'chunk_type': b'IHDR', 'start': 8, 'end': 33, 'length': 25}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 33, 'end': 8237, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 8237, 'end': 16441, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 16441, 'end': 24645, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 24645, 'end': 32849, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 32849, 'end': 41053, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 41053, 'end': 49257, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 49257, 'end': 57461, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 57461, 'end': 65665, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 65665, 'end': 73869, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 73869, 'end': 82073, 'length': 8204}
{'t

In [5]:
# notice that there are extra data at the end of the png file

print(f'len(sample_charcard_face_0_bytes)= {len(sample_charcard_face_0_bytes)}')
print(f'sample_charcard_face_0_chunk_data_list[-1][\'end\']= {sample_charcard_face_0_chunk_data_list[-1]["end"]}')

if len(sample_charcard_face_0_bytes) > sample_charcard_face_0_chunk_data_list[-1]['end']:
    print('extra data at the end of the png file')
else:
    print('no extra data at the end of the png file')


len(sample_charcard_face_0_bytes)= 179279
sample_charcard_face_0_chunk_data_list[-1]['end']= 84674
extra data at the end of the png file


In [6]:
# get the extra bytes from the char card file
sample_charcard_face_0_extra_bytes = sample_charcard_face_0_bytes[sample_charcard_face_0_chunk_data_list[-1]['end']:]

# write the extra bytes to a file
write_bytes('sample_charcard_face_0_extra_bytes.bin', sample_charcard_face_0_extra_bytes)

In [19]:
offset0 = 0

# 0: header
assert(sample_charcard_face_0_extra_bytes[offset0:offset0+4] == b'\x64\x00\x00\x00')
offset0 += 4

# 1: data: contains char "HCChara"
print(f'data.1.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+1], 'little')
offset0 += 1
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
print(bytes.decode('utf-8'))

# 2: data: contains version?
print(f'data.2.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+1], 'little')
offset0 += 1
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
print(bytes.decode('utf-8'))

# 3: data: contains png data
print(f'data.3.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.3.png'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

# 4: data
print(f'data.4.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.4.bin'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

# 5: data
print(f'data.5.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.5.bin'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

# 6: footer
print(f'data.6.offset = {hex(offset0)}')
offset0 += 4

print(f'offset0 = {offset0}')
print(f'len(sample_charcard_face_0_extra_bytes) = {len(sample_charcard_face_0_extra_bytes)}')


data.1.offset = 0x4
【HCChara】
data.2.offset = 0x12
0.0.0
data.3.offset = 0x18
save to sample_charcard_face_0_extra_bytes.3.png
data.4.offset = 0x60dd
save to sample_charcard_face_0_extra_bytes.4.bin
data.5.offset = 0x6253
save to sample_charcard_face_0_extra_bytes.5.bin
data.6.offset = 0x17189
offset0 = 94605
len(sample_charcard_face_0_extra_bytes) = 94605


In [39]:
def read_a_bytes(bytes, offset):
    length = int.from_bytes(bytes[offset:offset+1], 'little')
    assert(length > 0xa0)
    length -= 0xa0
    offset += 1
    v = bytes[offset:offset+length]
    print(f'offset = {hex(offset)}, bytes = {v}')
    offset += length
    return offset, v

def read_unknown_bytes(bytes, offset, length):
    v = bytes[offset:offset+length]
    print(f'offset = {hex(offset)}, unknown bytes = {v.hex()}')
    offset += length
    return offset, v



In [40]:
# parse sample_charcard_face_0_extra_bytes.4.bin
ex4_bytes = read_bytes('sample_charcard_face_0_extra_bytes.4.bin')
offset = 0

offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 2)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 4)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 3)
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 4)


offset = 0x0, unknown bytes = 81
offset = 0x2, bytes = b'lstInfo'
offset = 0x9, unknown bytes = 9884
offset = 0xc, bytes = b'name'
offset = 0x11, bytes = b'Custom'
offset = 0x18, bytes = b'version'
offset = 0x20, bytes = b'0.0.0'
offset = 0x26, bytes = b'pos'
offset = 0x29, unknown bytes = 00
offset = 0x2b, bytes = b'size'
offset = 0x2f, unknown bytes = cd0c6484
offset = 0x34, bytes = b'name'
offset = 0x39, bytes = b'Coordinate'
offset = 0x44, bytes = b'version'
offset = 0x4c, bytes = b'0.0.0'
offset = 0x52, bytes = b'pos'
offset = 0x55, unknown bytes = cd0c64
offset = 0x59, bytes = b'size'
offset = 0x5d, unknown bytes = cdfbc884
